In [1]:
%load_ext autoreload
%autoreload 2

In [46]:
import sys

sys.path.append('..')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mrmr import mrmr_classif

from src.machinelearning import (
    ANNClassifier,
    evaluate_ann,
    evaluate_sklearn,
    get_predictions,
    train_ann,
    train_decisiontree,
    train_skoperules,
    train_xgboost
)
from src.ml2rules import TreeRuler, ml2tree, sample_from_df
from src.utils import get_cc_mat, get_dataset, non_stratify_split, stratify_split

In [3]:
def plot_cc_ranges(df: pd.DataFrame) -> None:
    # Change plot style
    plt.style.use("ggplot")
    # Calculate the 1st and 3rd quartiles of df values
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    # Plot the mean of df values
    plt.figure(figsize=(10, 3))
    plt.barh(df.columns[::-1], df.mean()[::-1])
    plt.axvline(x=0, color="black", linestyle="-")
    plt.plot(q1, df.columns, "|", color="black", label="Q1", alpha=1)
    plt.plot(q3, df.columns, "|", color="black", label="Q3", alpha=1)
    for i, enzyme in enumerate(df.columns):
        plt.plot([q1[i], q3[i]], [enzyme, enzyme], color="black")
    plt.title("CC_XTR")
    plt.grid()
    plt.show()
    
def get_value_counts(df: pd.DataFrame) -> None:
    display(
        pd.DataFrame(
        data=[df['label'].value_counts(normalize=True).round(4), df['label'].value_counts()],
        index=["percentage", "absolute"]    
    ).T
    )


### Data loading

In [ ]:
df = get_dataset(
    labels_file="../data/class_vector_train_ref.mat",
    params_file="../data/training_set_ref.mat",
    names_file="../data/paremeterNames.mat",
)

print(f"***Dataset shape: {df.shape}\n")

# Load FCC data
enzyme, commonEnz, allEnzymes, \
    commonConCoeff, allConCoeff = get_cc_mat("../data/ccXTR_ref.mat")

plot_cc_ranges(commonConCoeff)

In [ ]:
# Create dataset for HXK enzyme; label 1 if CC < 0, 0 otherwise
idx_HXK = commonConCoeff[commonConCoeff["HXK"] < 0].index

df_HXK = df.drop("label", axis=1)
df_HXK["label"] = [1 if i in idx_HXK else 0 for i in df_HXK.index]
df_HXK["label"].value_counts(normalize=True).round(4)
df = df_HXK.copy()

# Create a dataframe with value_counts both percentage and absolute
display(
    pd.DataFrame(
    data=[df['label'].value_counts(normalize=True).round(4), df['label'].value_counts()],
    index=["percentage", "absolute"]    
).T
)

del df_HXK

### Train - Test data split

In [ ]:
# Split data
# X_train, X_test, y_train, y_test = non_stratify_split(
#     data=df, train_size=100000, target="label", zero_class_pct=0.60
# )

X_train, X_test, y_train, y_test = stratify_split(
    data=df, train_size=0.5, target="label"
)

train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

print('Training set value counts:')
get_value_counts(train_df)

print('Test set value counts:')
get_value_counts(test_df)

### Feature Selection

In [ ]:
K = int(df.drop("label", axis=1).shape[1] / 4)
print(f"***Number of features to select: {K}")
selected_features = mrmr_classif(X=df.drop("label", axis=1), y=df["label"], K=K)

# keep only selected features
X_train = X_train[selected_features]
X_test = X_test[selected_features]
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

print(f'Training set shape: {train_df.shape}')
print(f'Test set shape: {test_df.shape}')

### Machine Learning

##### Decision Tree - iSHRUNCK

In [ ]:
cart_model = train_decisiontree(X_train, y_train, scoring='accuracy', n_trials=1)

In [ ]:
cart_model

In [ ]:
# Evaluate model
print('*** Evaluation on training set:')
evaluate_sklearn(cart_model, X_train, y_train)

print()

print('*** Evaluation on test set:')
evaluate_sklearn(cart_model, X_test, y_test)

In [ ]:
ruler = TreeRuler(df=train_df, tree_clf=cart_model, target="label")
ruler.get_rules()
ruler.rules

In [ ]:
rule = ruler.get_rule_constraints(0)
sampled_df = sample_from_df(test_df, rule)
print(f'Number of parameter sets following the rule: {sampled_df.shape[0]}')
get_value_counts(sampled_df)

In [ ]:
_ = test_df[test_df['ASN@pi_m'] <= 0.63]
_ = _[_['XRI@nadh_c'] <= 0.604]
_ = _[_['TPI@t3p_c'] > 0.421]
_ = _[_['XRI@nad_c'] > 0.611]
_ = _[_['PGK@dpg_c'] <= 0.909]
_ = _[_['XDH@xlt_c'] <= 0.691]
_ = _[_['GLYCt@glyc_c'] <= 0.936]

get_value_counts(_)

##### Skope - Rules

In [ ]:
skope_rules_clf = train_skoperules(X_train, y_train, scoring='matthews_corrcoef', n_iter=25)

In [ ]:
skope_rules_clf

In [ ]:
# Print skope-rules rules
print(f'*** Number of rules {len(skope_rules_clf.rules_)}')

# Get rules
for i in skope_rules_clf.rules_:
    print(i)

In [ ]:
_ = test_df[test_df['XRI@nadh_c'] <= 0.7392323613166809]
_ = _[_['ASN@pi_m'] > 0.632788211107254]
_ = _[_['ASN@atp_m'] <= 0.8232964277267456]

get_value_counts(_)

##### XGBoost

In [ ]:
xgb_model = train_xgboost(X_train, y_train, scoring='accuracy', n_trials=25)

In [ ]:
xgb_model

In [ ]:
# Evaluate model
print('*** Evaluation on training set:')
evaluate_sklearn(xgb_model, X_train, y_train)

print()

print('*** Evaluation on test set:')
evaluate_sklearn(xgb_model, X_test, y_test)

In [ ]:
xgb_preds = xgb_model.predict(X_train)
xgb2tree = ml2tree(X_train, xgb_preds, n_trial=25)

In [ ]:
xgb2tree

In [ ]:
evaluate_sklearn(xgb2tree, X_test, y_test)

In [ ]:
ruler = TreeRuler(df=train_df, tree_clf=xgb2tree, target="label")
ruler.get_rules()
ruler.rules

##### Neural Network

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net = ANNClassifier(input_dim=X_train.shape[1], hidden_dim=2048,
                    output_dim=1, hidden_layers=4).to(device)
net = train_ann(net, X_train, y_train, num_epochs=1000, learning_rate=0.001, batch_size=2048)

In [ ]:
# Evaluate model
evaluate_ann(net, X_test, y_test)

In [ ]:
ann_preds = get_predictions(net, X_train)
ann2tree = ml2tree(X_train, ann_preds, n_trials=200)

In [ ]:
ann2tree

In [ ]:
evaluate_sklearn(ann2tree, X_test, y_test)

In [ ]:
ruler = TreeRuler(df=train_df, tree_clf=ann2tree, target="label")
ruler.get_rules()
ruler.rules

In [ ]:
_ = test_df[test_df['ASN@pi_m'] > 0.679]
_ = _[_['XRI@nadh_c'] <= 0.633]
_ = _[_['ASN@atp_m'] <= 0.573]

get_value_counts(_)

### Exlainable Machine Learning